In [ ]:
# pip install datasets --user

In [1]:
from run_train import create_model_and_diffusion
from utils.step_sample import create_named_schedule_sampler
from train import TrainLoop
from utils.data import load_data_text
from tokenizer import load_tokenizer, load_model_emb
from sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch, os
from utils import dist_util
from functools import partial
import pickle
import random

2024-02-12 11:44:54.848429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [114]:
# with open('vocab_list.pickle', 'rb') as handle:
#     vocab_list = pickle.load(handle)
# vocab_list = list(vocab_list.values())

In [4]:
dist_util.clear_cache()

In [5]:
lr=0.0001
batch_size=10
microbatch=5
epochs=5000
eval_interval=1000
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=1200
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=102
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [6]:
cc_data_dir='data/commonsense'
ss_data_dir='data/shakespeare'
ss_small_data_dir='data/mini-shakespeare'
combined_data_dir='data/combined'
regular_data_dir='data'

# set the data directory
data_dir=regular_data_dir

In [7]:
set_seed(seed)

In [8]:
tokenizer = load_tokenizer('shakespeare', config_name)

In [9]:
tokenizer.vocab_size

30474

In [10]:
tokenizer.encode_token('find we a time for fright peace to pant')

[2, 611, 139, 26, 413, 121, 3021, 649, 94, 7338, 3]

In [11]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [12]:
model_weight

Embedding(30474, 128)

In [13]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size

In [14]:
vocab_size

30474

In [15]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ["my name is mary, sir. good mistress mary accost,-- you mistake, knight, 'accost' is front her, board", 'if it be possible for you to displace it with your little finger, there is some hope the ladies of rome, especially his mother, may prevail with him.'] ["her, woo her, assail her. by my troth, i would not undertake her in this company. is that the meaning of 'accost'?", "but i say there is no hope in't: our throats are sentenced and stay upon execution. is't possible that so short a time can alter the"]
RAM used: 2310.20 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 110814
})
RAM used: 2348.33 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/110814 [00:00<?, ? examples/s]

Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/xykong/.local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/xykong/.local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xykong/.local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/xykong/.local/lib/python3.9/site-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/xykong/.local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xykong/.local/lib/python3.9/site-packages/datasets/utils/py_utils.py", line 614, in _write_generator_to_queue
    for i, result in enumerate(func(**kwargs)):
Tracebac

TimeoutError: 

Passed in as batch in TrainLoop - this is the batch data

In [128]:
# next(data)[0].shape # batch_size, seq_len, hidden_dim

In [129]:
# next(data)[0]

Passed in as cond in TrainLoop - this is a dictionary of input_ids and input_mask

In [130]:
# next(data)[1]

In [131]:
# next(data)[1]['input_ids'].shape # batch_size, hidden_dim

In [132]:
# next(data)[1]['input_mask'].shape # batch_size, hidden_dim

In [133]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [134]:
model.to(dist_util.dev())

TransformerNetModel(
  (word_embedding): Embedding(30474, 128)
  (lm_head): Linear(in_features=128, out_features=30474, bias=True)
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=768, bias=True)
  )
  (input_up_proj): Sequential(
    (0): Linear(in_features=128, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
  (input_transformers): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768,

In [135]:
pytorch_total_params = sum(p.numel() for p in model.parameters())

In [136]:
pytorch_total_params

91219082

In [137]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [138]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=batch_size,
        microbatch=microbatch,
        lr=lr,
        ema_rate=ema_rate,
        schedule_sampler=schedule_sampler,
        weight_decay=weight_decay,
        epochs=epochs,
#         eval_data=data_valid,
        eval_interval=eval_interval
    ).run_loop()



======== Training starts now ========


Epoch 0/5000 Loss: 1.1900126934051514
Epoch 1/5000 Loss: 0.9100378751754761
Epoch 2/5000 Loss: 0.8300307989120483
Epoch 3/5000 Loss: 0.769633412361145
Epoch 4/5000 Loss: 0.751349687576294
Epoch 5/5000 Loss: 0.8101098537445068
Epoch 6/5000 Loss: 0.7840892672538757
Epoch 7/5000 Loss: 0.819953203201294
Epoch 8/5000 Loss: 0.7605277299880981
Epoch 9/5000 Loss: 0.7890160083770752
Epoch 10/5000 Loss: 0.7736339569091797
Epoch 11/5000 Loss: 0.7514801621437073
Epoch 12/5000 Loss: 0.7526057362556458
Epoch 13/5000 Loss: 0.6375998258590698
Epoch 14/5000 Loss: 0.6373530626296997
Epoch 15/5000 Loss: 0.6963253617286682
Epoch 16/5000 Loss: 0.6082180738449097
Epoch 17/5000 Loss: 0.5555065870285034
Epoch 18/5000 Loss: 0.5474282503128052
Epoch 19/5000 Loss: 0.567943811416626
Epoch 20/5000 Loss: 0.5037021040916443
Epoch 21/5000 Loss: 0.524573802947998
Epoch 22/5000 Loss: 0.5013637542724609
Epoch 23/5000 Loss: 0.4826151132583618
Epoch 24/5000 Loss: 0.489516794681549

In [177]:
# pickle.dump(model, open("models/model_0212.pkl", 'wb'))
# pickle.dump(diffusion, open("models/diffusion_0212.pkl", 'wb'))

In [2]:
# with open('models/model_0212.pkl', 'rb') as handle:
#     model = pickle.load(handle)

In [3]:
# with open('models/diffusion_0212.pkl', 'rb') as handle:
#     diffusion = pickle.load(handle)

# Finetuning on only Shakespeare

In [46]:
# model.eval().to(dist_util.dev())

# model_emb = torch.nn.Embedding(
#         num_embeddings=tokenizer.vocab_size, 
#         embedding_dim=hidden_dim, 
#         _weight=model.word_embedding.weight.clone().cpu()
#     ).eval()

In [47]:
# ss_data = load_data_text(
#         batch_size=10,
#         seq_len=seq_len,
#         data_dir=ss_data_dir,
#         loaded_vocab=tokenizer,
#         model_emb=model_emb.cpu() # use model's weights as init
#     )

In [48]:
# model.train() # TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

# TrainLoop(
#         model=model,
#         diffusion=diffusion,
#         data=ss_data,
#         batch_size=batch_size,
#         microbatch=microbatch,
#         lr=lr,
#         ema_rate=ema_rate,
#         schedule_sampler=schedule_sampler,
#         weight_decay=weight_decay,
#         epochs=epochs,
# #         eval_data=data_valid,
#         eval_interval=eval_interval
#     ).run_loop()

# Generating sequences

In [16]:
import numpy as np
device = dist_util.dev()

def sampling(
    model, 
    diffusion, 
    tokenizer, 
    device=device, 
    batch_size=16, 
    seq_len=128, 
    data_dir='data/',
    split='test',
    clip_denoised=False, 
    model_kwargs={}, 
    top_p=0, 
    step_gap=1,
    clamp_step=0):
    
    # ---- putting the model into eval mode ----
    model.eval().requires_grad_(False).to(device)
    
    hidden_dim = model.word_embedding.embedding_dim

    model_emb = torch.nn.Embedding(
            num_embeddings=tokenizer.vocab_size, 
            embedding_dim=hidden_dim, 
            _weight=model.word_embedding.weight.clone().cpu()
        ).eval().requires_grad_(False)
    
    # ---- getting test data ----

    data_test = load_data_text(
            batch_size=batch_size,
            seq_len=seq_len,
            deterministic=True,
            data_dir=data_dir,
            split=split,
            loaded_vocab=tokenizer,
            model_emb=model_emb.cpu(),  # using the same embedding wight with tranining data
            loop=False
        )

    all_test_data = []

    try:
        while True:
            batch, cond = next(data_test)
            # print(batch.shape)
            all_test_data.append(cond)

    except StopIteration:
        print('### End of reading iteration...')

    model_emb.to(device)
    
    # ---- iterating through the test data to generate sequences ----
    iterator = iter(all_test_data)
    word_lst_recover = []
    word_lst_ref = []
    word_lst_source = []

    for cond in iterator:

        input_ids_x = cond.pop('input_ids').to(device)
        x_start = model.get_embeds(input_ids_x)
        input_ids_mask = cond.pop('input_mask')
        input_ids_mask_ori = input_ids_mask

        noise = torch.randn_like(x_start)
        input_ids_mask = torch.broadcast_to(input_ids_mask.unsqueeze(dim=-1), x_start.shape).to(device)
        x_noised = torch.where(input_ids_mask == 0, x_start, noise)

        model_kwargs = {}
        sample_fn = diffusion.p_sample_loop
        sample_shape = (x_start.shape[0], seq_len, hidden_dim)

        samples = sample_fn(
            model,
            sample_shape,
            noise=x_noised,
            clip_denoised=clip_denoised,
            denoised_fn=partial(denoised_fn_round, model_emb),
            model_kwargs=model_kwargs,
            top_p=top_p,
            clamp_step=clamp_step,
            clamp_first=True,
            mask=input_ids_mask,
            x_start=x_start,
            gap=step_gap
        )

        # print(samples[0].shape) # samples for each step

        sample = samples[-1]

        # print('decoding for seq2seq', )
        # print(sample.shape)

        logits = model.get_logits(sample)  # bsz, seqlen, vocab
        cands = torch.topk(logits, k=1, dim=-1)

        for seq, input_mask in zip(cands.indices, input_ids_mask_ori):
            len_x = seq_len - sum(input_mask).tolist()
            tokens = tokenizer.decode_token(seq[len_x:])
            word_lst_recover.append(tokens)

        for seq, input_mask in zip(input_ids_x, input_ids_mask_ori):
            # tokens = tokenizer.decode_token(seq)
            len_x = seq_len - sum(input_mask).tolist()
            word_lst_source.append(tokenizer.decode_token(seq[:len_x]))
            word_lst_ref.append(tokenizer.decode_token(seq[len_x:]))
        break # after 1 batch
    return word_lst_source, word_lst_recover, word_lst_ref


def get_efficient_knn(model_emb, text_emb):
    emb_norm = (model_emb**2).sum(-1).view(-1, 1) # vocab
    text_emb_t = torch.transpose(text_emb.view(-1, text_emb.size(-1)), 0, 1) # d, bsz*seqlen
    arr_norm = (text_emb ** 2).sum(-1).view(-1, 1) # bsz*seqlen, 1
    # print(emb_norm.shape, arr_norm.shape)
    dist = emb_norm + arr_norm.transpose(0, 1) - 2.0 * torch.mm(model_emb, text_emb_t) # (vocab, d) x (d, bsz*seqlen)
    dist = torch.clamp(dist, 0.0, np.inf)
    # print(dist.shape)
    topk_out = torch.topk(-dist, k=1, dim=0)
    return topk_out.values, topk_out.indices

def denoised_fn_round(model, text_emb, t):
    # print(text_emb.shape) # bsz, seqlen, dim
    model_emb = model.weight  # input_embs
    # print(t)
    old_shape = text_emb.shape
    old_device = text_emb.device

    if len(text_emb.shape) > 2:
        text_emb = text_emb.reshape(-1, text_emb.size(-1))
    else:
        text_emb = text_emb
    # val, indices = get_knn(model_emb, text_emb.to(model_emb.device), dist=dist)
    val, indices = get_efficient_knn(model_emb, text_emb.to(model_emb.device))
    rounded_tokens = indices[0]
    # print(rounded_tokens.shape)
    new_embeds = model(rounded_tokens).view(old_shape).to(old_device)

    return new_embeds

In [17]:
data_dir

'data'

In [18]:
clip_denoised = False
top_p = 0
clamp_step = 0

In [25]:
word_lst_source, word_lst_recover, word_lst_ref = sampling(model, diffusion, tokenizer, data_dir=data_dir, batch_size=5, split='test_custom', seq_len=20)

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['the meaning of life is', 'call her forth to me .'] ['', '']
RAM used: 2649.74 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 3
})
RAM used: 2649.74 MB


Running tokenizer on dataset (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 3
})
### tokenized_datasets...example [2, 83, 2833, 100, 476, 126, 3]
RAM used: 2651.78 MB


merge and mask:   0%|          | 0/3 [00:00<?, ? examples/s]

RAM used: 2651.83 MB


padding:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 3
}) padded dataset
RAM used: 2651.84 MB
RAM used: 2651.84 MB
### End of reading iteration...


  0%|          | 0/1200 [00:00<?, ?it/s]

Generating 20 sentences takes 5 minutes

In [26]:
word_lst_source

['[CLS] the meaning of life is [SEP] [SEP]',
 '[CLS] call her forth to me [UNK] [SEP] [SEP]',
 '[CLS] what is your will? [SEP] [SEP]']

In [27]:
word_lst_recover

['ten barg their france noble is lightest [CLS] unwilling pace [CLS] our',
 'grace my tymbriaoras hereirrel latin prithee on enter [CLS]',
 'have tribute some king keep she who obsequiously she [CLS] enter to']

In [22]:
word_lst_ref

['[CLS] [SEP]', '[CLS] [SEP]', '[CLS] [SEP]']